<a href="https://colab.research.google.com/github/SunkeerthM/AlgoBook/blob/master/Plant_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import glob
import pickle

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tensorflow-gpu

     |████████████████████████████████| 454.3MB 33kB/s 


In [9]:
!nvidia-smi


NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [9]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [10]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [11]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/My Drive/Programs/Datasets/train'
valid_path = '/content/drive/My Drive/Programs/Datasets/test'

In [13]:
inception = InceptionV3(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)

In [14]:
for layer in inception.layers:
  layer.trainable = False

In [15]:
folders = glob('/content/drive/My Drive/Programs/Datasets/train/*')

In [16]:
folders

['/content/drive/My Drive/Programs/Datasets/train/Tomato___Bacterial_spot',
 '/content/drive/My Drive/Programs/Datasets/train/Tomato___healthy',
 '/content/drive/My Drive/Programs/Datasets/train/Tomato___Early_blight',
 '/content/drive/My Drive/Programs/Datasets/train/Tomato___Leaf_Mold',
 '/content/drive/My Drive/Programs/Datasets/train/Tomato___Target_Spot',
 '/content/drive/My Drive/Programs/Datasets/train/Tomato___Tomato_mosaic_virus',
 '/content/drive/My Drive/Programs/Datasets/train/Tomato___Septoria_leaf_spot',
 '/content/drive/My Drive/Programs/Datasets/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 '/content/drive/My Drive/Programs/Datasets/train/Tomato___Spider_mites Two-spotted_spider_mite',
 '/content/drive/My Drive/Programs/Datasets/train/Tomato___Late_blight']

In [17]:
 x = Flatten()(inception.output)

In [18]:
prediction = Dense(len(folders), activation='softmax')(x)

In [19]:
#create a model objet
model = Model(inputs = inception.input, outputs = prediction)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
______________________________________________________________________________________________

In [21]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [23]:
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Programs/Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 18355 images belonging to 10 classes.


In [24]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Programs/Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 4585 images belonging to 10 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1148/1148 [==============================] - 5699s 5s/step - loss: 4.3930 - accuracy: 0.6745 - val_loss: 4.2401 - val_accuracy: 0.7333
Epoch 2/10
1148/1148 [==============================] - 2634s 2s/step - loss: 3.3340 - accuracy: 0.7946 - val_loss: 3.7038 - val_accuracy: 0.7812
Epoch 3/10
1148/1148 [==============================] - 2613s 2s/step - loss: 3.3613 - accuracy: 0.8163 - val_loss: 3.8819 - val_accuracy: 0.8089
Epoch 4/10
1148/1148 [==============================] - 2617s 2s/step - loss: 3.1361 - accuracy: 0.8389 - val_loss: 2.9308 - val_accuracy: 0.8454
Epoch 5/10
1148/1148 [==============================] - 2616s 2s/step - loss: 3.2059 - accuracy: 0.8475 - val_loss: 3.5377 - val_accuracy: 0.8321
Epoch 6/10
1148/1148 [==============================] - 2617s 2s/step - loss: 2.9558 - accuracy: 0.8634 - val_loss: 4.8547 - val_accuracy: 0.8329
Epoch 7/10
1148/1148 [==============================] - 2629s 2s/step - loss: 2.8870 - accuracy: 0.8702 - val_loss: 4.2352 -

In [1]:
import matplotlib.pyplot as plt

In [2]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: ignored